In [ ]:
# Source: https://stackoverflow.com/questions/62216146/data-scraping-from-vivino-com
from bs4 import BeautifulSoup
import requests
import re

data_structure = ["Bottle Price",	"Winery",	"Year", "Wine Style",	"Region",	"Grape Varieties",	"ABV",	"Dry/Sweet",	"Tannin",	"Soft/Acidic",	"Flavor Profile",	"Pairings", "Rating", "Category", "Light/Bold", "Profile Specifics", "Fizziness", "Country"]


def get_data(vivino_url):
    if(vivino_url == "manual"):
      return(data_structure)
    data_dict = ["Bottle Price",	"Winery",	"Year", "Wine Style",	"Region",	"Grape Varieties",	"ABV",	"Dry/Sweet",	"Tannin",	"Soft/Acidic",	"Flavor Profile",	"Pairings", "Rating", "Category", "Light/Bold", "Profile Specifics", "Fizziness", "Country"]

    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "Accept-Language": "en-US", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    r = requests.get(vivino_url, headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content, "html.parser")
    #print(soup.prettify)

    for d in soup.find_all('td', attrs={'class':'wineFacts__fact--3BAsi'}):
      factSearch = re.search(r'href="/(.*)/([^"]+)".*?>(.*?)</a>', str(d))
      if(factSearch == None):
        abvSearch = re.search(r'<span>(.*?)%</span>', str(d))
        if(abvSearch == None):
          continue
        else:
          data_dict[6] = abvSearch.group(1)
        continue
      if(factSearch.group(1) == "wineries"):
        data_dict[1] = factSearch.group(3)
      elif(factSearch.group(1) == "wine-regions"):
        data_dict[4] = factSearch.group(3)
      elif(factSearch.group(1) == "wine-styles"):
        data_dict[3] = factSearch.group(3)
      elif(factSearch.group(1) == "grapes"):
        data_dict[5] = factSearch.group(3)
      else:
        grapeSearch = re.findall(r'>(.*?)</a>', str(factSearch.group(1)))
        if(grapeSearch == None):
          continue
        grapes = []
        for g in grapeSearch:
          grapes.append(g)

        if(len(grapes) != 0):
          data_dict[5] = str(grapes)

    pairingList = []
    for d in soup.find_all('div', attrs={'class':'foodPairing__foodImage--2OYHg'}):
      pairingItem = re.search(r'label="(.*?)"', str(d))
      if(pairingItem == None):
        continue
      pairingList.append(pairingItem.group(1))
    if(pairingList != []):
      data_dict[11] = str(pairingList)

    wineId = re.search(r'"wine":{"id":(.*?),"', str(soup))
    if(wineId == None):
      print("ID not found")
      return []

    JSONheaders = {'authority': 'www.vivino.com', 'method': 'GET', 'scheme': 'https', 'accept': 'application/json', 'accept-encoding': 'gzip, deflate, br, zstd', 'accept-language': 'en-US,en;q=0.9', 'content-type': 'application/json', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36' } #GPT provided
    tasteReq = requests.get("https://www.vivino.com/api/wines/" + wineId.group(1) + "/tastes?language=en", headers=JSONheaders)
    if tasteReq.status_code == 200:
      tasting_notes_data = tasteReq.json()

      if(tasting_notes_data['tastes']['structure'] != None):
        data_dict[9] = (tasting_notes_data['tastes']['structure']['acidity'] if tasting_notes_data['tastes']['structure']['acidity'] != None else 0)
        data_dict[9] = (tasting_notes_data['tastes']['structure']['acidity'] if tasting_notes_data['tastes']['structure']['acidity'] != None else 0)
        data_dict[16] = (tasting_notes_data['tastes']['structure']['fizziness'] if tasting_notes_data['tastes']['structure']['fizziness'] != None else 0)
        data_dict[14] = (tasting_notes_data['tastes']['structure']['intensity'] if tasting_notes_data['tastes']['structure']['intensity'] != None else 0)
        data_dict[7] = (tasting_notes_data['tastes']['structure']['sweetness'] if tasting_notes_data['tastes']['structure']['sweetness'] != None else 0)
        data_dict[8] = (tasting_notes_data['tastes']['structure']['tannin'] if tasting_notes_data['tastes']['structure']['tannin'] != None else 0)

      top3 = {"x": -1, "y": -1, "z": -1}
      top5 = {"v": -1, "w": -1, "x": -1, "y": -1, "z": -1}

      if(tasting_notes_data['tastes']['flavor'] != None):
        for groupJSON in tasting_notes_data['tastes']['flavor']:
          group, mentions = groupJSON['group'], groupJSON['stats']['count']
          if(mentions > min(top3.values()) and not group in top3):
            #pop the minimum and add yourself
            top3.pop([minKey for minKey, count in top3.items() if count == min(top3.values())][0])
            top3[group] = mentions

          if 'primary_keywords' in groupJSON:
            for primaryJSON in groupJSON['primary_keywords']:
              primary, count = primaryJSON['name'], primaryJSON['count']
              #print(primary, count)
              if(count > min(top5.values()) and not primary in top5):
                top5.pop([minKey for minKey, pCount in top5.items() if pCount == min(top5.values())][0])
                top5[primary] = count

      top3.pop('x', None)
      top3.pop('y', None)
      top3.pop('z', None)
      top5.pop('v', None)
      top5.pop('w', None)
      top5.pop('x', None)
      top5.pop('y', None)
      top5.pop('z', None)

      if(top3 != {}):
        data_dict[10] = str(list(top3))

      if(top5 != {}):
        data_dict[15] = str(list(top5))

    else:
      print(f"failed w {r.status_code}")

    '''priceReq = requests.get("https://www.vivino.com/api/wines/" + wineId.group(1) + "/checkout_prices?language=en", headers=JSONheaders)
    print("https://www.vivino.com/api/wines/" + wineId.group(1) + "/checkout_prices?language=en")
    if priceReq.status_code == 200:
      price_data = priceReq.json()
      print(price_data)
      if(price_data['checkout_prices'] != [] and price_data['checkout_prices'][0]['market'] != [] and price_data['checkout_prices'][0]['market']['currency']['code'] == "USD"):
        data_dict[0] = price_data['checkout_prices'][0]['availability']['median']['amount']

    else:
      print(f"failed w {r.status_code}")
'''
    priceSearch = re.search(r'prices_and_availability.*"median":{"amount":(.*?),"type".*USD', str(soup))
    if(priceSearch != None):
      data_dict[0] = priceSearch.group(1)

    for d in soup.find_all('a', attrs={'class':'anchor_anchor__m8Qi- breadCrumbs__link--1TY6b'}):
      breadcrumb = re.search(r'href="/(.*?)/.*>(.*?)</a>', str(d))
      typeRe = re.search(r'wine_type_ids.*>(.*?)</a>', str(d))
      if(typeRe != None):
        data_dict[13] = typeRe.group(1)
      if(breadcrumb != None):
        if(breadcrumb.group(1) == "wine-countries"):
          data_dict[17] = breadcrumb.group(2)
        elif(breadcrumb.group(1) == "wine-regions" and data_dict[4] == "Region"):
          data_dict[4] = breadcrumb.group(2)
        elif(breadcrumb.group(1) == "wineries" and data_dict[1] == "Winery"):
          data_dict[1] = breadcrumb.group(2)

    if(data_dict[2] == "Year"):
      year = re.search(r'year=([^&]*)', vivino_url)
      if(year != None):
        data_dict[2] = year.group(1)

    rating = re.search(r'href=.*min_rating=(.*?)&', str(soup))
    if(rating != None and rating.group(1) != "1"):
      data_dict[12] = rating.group(1)

    notFound = 0
    notFoundItems = []
    for i in range (len(data_structure)):
      if(data_structure[i] == data_dict[i]):
        notFound += 1
        notFoundItems.append(data_dict[i])
        data_dict[i] = ""

    if(notFound != 0):
      print(f"{notFound} item(s) not found; {notFoundItems}", end = "")

    return data_dict

get_data("https://www.vivino.com/US/en/messina-hof-nimitz-old-texas-roundup-dry-red-blend/w/13116715?srsltid=AfmBOop6FZjTWtE2vp4vYh6z50l_rrE_gQ11hdGfrkAb1inzgvlz2fYt")

5 item(s) not found; ['Bottle Price', 'Year', 'Flavor Profile', 'Rating', 'Profile Specifics']

['',
 'Messina Hof',
 '',
 'Texas Red',
 'Texas',
 "['Shiraz/Syrah', 'Cabernet Sauvignon', 'Malbec', 'Petite Sirah']",
 '14.5',
 1.5,
 4,
 3,
 '',
 "['Beef', 'Lamb', 'Game (deer, venison)']",
 '',
 'Red wine',
 4,
 '',
 0,
 'United States']

In [ ]:
import gspread
from google.colab import auth
from google.auth import default
import time

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
from os import read

def updateTasteMapping(vivino_url, tasteMappings):
  headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "Accept-Language": "en-US", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
  r = requests.get(vivino_url, headers=headers)#, proxies=proxies)
  content = r.content
  soup = BeautifulSoup(content, "html.parser")
  wineId = re.search(r'"wine":{"id":(.*?),"', str(soup))

  JSONheaders = {'authority': 'www.vivino.com', 'method': 'GET', 'scheme': 'https', 'accept': 'application/json', 'accept-encoding': 'gzip, deflate, br, zstd', 'accept-language': 'en-US,en;q=0.9', 'content-type': 'application/json', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36' } #GPT provided
  tasteReq = requests.get("https://www.vivino.com/api/wines/" + wineId.group(1) + "/tastes?language=en", headers=JSONheaders)
  if tasteReq.status_code == 200:
    tasting_notes_data = tasteReq.json()
    if(tasting_notes_data['tastes']['flavor'] != None):
      for groupJSON in tasting_notes_data['tastes']['flavor']:
        if('primary_keywords' not in groupJSON):
          continue
        group, primary = groupJSON['group'], groupJSON['primary_keywords']
        if(primary == None):
          continue
        for primaryJSON in primary:
           primaryTaste = primaryJSON['name']
           tasteMappings[group].add(primaryTaste)

  else:
    print(f"failed w {r.status_code}")

def readAllTasteMappings():
  tasteMappings = {'earth': set(), 'floral': set(), 'microbio': set(), 'tropical_fruit': set(), 'citrus_fruit': set(), 'dried_fruit': set(), 'tree_fruit': set(), 'oak': set(), 'vegetal': set(), 'black_fruit': set(), 'non_oak': set(), 'red_fruit': set(), 'spices': set()}

  spreadsheet = gc.open('WineList')
  worksheet = spreadsheet.get_worksheet(0) #0 for real data, 1 for dummy
  rows = worksheet.get_all_values()

  for r in range (1, len(rows)):
    if(rows[r][0] == "vivino" or rows[r][0] == "manual"):
      continue
    print(f"Processing row {r+1}... ", end = " ")
    updateTasteMapping(rows[r][0], tasteMappings)

  return tasteMappings

def updateWineIDs():
  spreadsheet = gc.open('WineList')
  worksheet = spreadsheet.get_worksheet(0) #0 for real data, 1 for dummy
  rows = worksheet.get_all_values()

  for r in range (1, len(rows)):
    if(rows[r][0] == "vivino" or rows[r][0] == "manual" or rows[r][22] != ""):
      continue
    url = rows[r][0]
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "Accept-Language": "en-US", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    req = requests.get(url, headers=headers)#, proxies=proxies)
    content = req.content
    soup = BeautifulSoup(content, "html.parser")
    wineId = re.search(r'"wine":{"id":(.*?),"', str(soup))
    print(url + " " + wineId.group(1))
    worksheet.update_cell(r+1, 23, wineId.group(1))

updateWineIDs()
#finalTasteMap = readAllTasteMappings()
#print(finalTasteMap)

In [ ]:
profile_options = set()
pairing_options = set()
category_list = set()
specific_tastes = set()

def update_profiles():
  spreadsheet = gc.open('WineList')
  worksheet = spreadsheet.get_worksheet(0) #0 for real data, 1 for dummy
  dummy_worksheet = spreadsheet.get_worksheet(1)
  rows = worksheet.get_all_values()
  dummy_rows = dummy_worksheet.get_all_values()

  for r in range (1, len(rows)):
    if(rows[r][0] == "vivino" or rows[r][5] != "" or rows[r][6] != "" or rows[r][0] == "manual"):
      continue
    print(f"Processing row {r+1}... ", end = " ")
    row_data = get_data(rows[r][0])
    worksheet.update([row_data], 'E' + str(r + 1) + ':V' + str(r + 1))
    print()

def update_profiles_LLM():
  spreadsheet = gc.open('WineList')
  worksheet = spreadsheet.get_worksheet(0) #0 for real data, 1 for dummy
  dummy_worksheet = spreadsheet.get_worksheet(1)
  rows = worksheet.get_all_values()
  dummy_rows = dummy_worksheet.get_all_values()

  for r in range (1, len(rows)):
    if(rows[r][0] == "vivino" or dummy_rows[r][5] != "" or dummy_rows[r][6] != ""):
      continue
    print(f"Processing row {r+1}... ", end = " ")
    row_data = get_data(rows[r][0])
    print()
    print(row_data)
    llm_data = fill_gaps(rows[r][1], row_data)
    print(llm_data)
    dummy_worksheet.update([llm_data], 'E' + str(r + 1) + ':V' + str(r + 1))
    print()
    time.sleep(2)

def remake_LLM(row):
  row = row-1
  spreadsheet = gc.open('WineList')
  worksheet = spreadsheet.get_worksheet(0) #0 for real data, 1 for dummy
  dummy_worksheet = spreadsheet.get_worksheet(1)
  rows = worksheet.get_all_values()
  dummy_rows = dummy_worksheet.get_all_values()

  if(rows[row][0] == "vivino"):
    return
  print(f"Processing row {row+1}... ", end = " ")
  row_data = get_data(rows[row][0])
  print()
  print(row_data)
  llm_data = fill_gaps(rows[row][1], row_data)
  print(llm_data)
  dummy_worksheet.update([llm_data], 'E' + str(row + 1) + ':V' + str(row + 1))
  print()
  time.sleep(2)

def read_profiles():
  spreadsheet = gc.open('WineList')
  worksheet = spreadsheet.get_worksheet(0) #0 for real data, 1 for dummy
  dummy_worksheet = spreadsheet.get_worksheet(1)
  rows = worksheet.get_all_values()
  dummy_rows = dummy_worksheet.get_all_values()

  for r in range (1, len(rows)):
    if(rows[r][0] == "vivino"):
      continue
    #print(f"Processing row {r+1}... ", end = " ")
    if(r == 72):
      print(rows[r])
    prof_search = re.findall(r"'(.*?)'", str(rows[r][14]))
    for p in prof_search:
      profile_options.add(p)

    pair_search = re.findall(r"'(.*?)'", str(rows[r][15]))
    for p in pair_search:
      pairing_options.add(p)

    pair_search2 = re.findall(r'"(.*?)"', str(rows[r][15]))
    for p in pair_search2:
      pairing_options.add(p)

    category_list.add(rows[r][17])

    spec_search = re.findall(r"'(.*?)'", str(rows[r][19]))
    for p in spec_search:
      specific_tastes.add(p)

def fix_LLM_mistakes():
  spreadsheet = gc.open('WineList')
  worksheet = spreadsheet.get_worksheet(0) #0 for real data, 1 for dummy
  dummy_worksheet = spreadsheet.get_worksheet(1)
  rows = worksheet.get_all_values()
  dummy_rows = dummy_worksheet.get_all_values()

  wrong_rows = 0
  unfixed_rows = 0

  for r in range (1, len(rows)):
    issue = False;
    for c in range(1, len(rows[0])):
      if(rows[r][c] != "" and rows[r][c] != dummy_rows[r][c]):
        #if price is not within 5 dollars
        if(c == 4 and abs(float(rows[r][c]) - float(dummy_rows[r][c])) > 5):
          issue = True;
          wrong_rows += 1
        #if the ratings it gives are not within .01
        elif((c == 11 or c == 12 or c == 13 or c == 18) and abs(float(rows[r][c]) - float(dummy_rows[r][c])) > .01):
          issue = True
          wrong_rows += 1
        elif(c != 14 and c != 15 and c != 19 and c != 4 and c != 11 and c != 12 and c != 13 and c != 18):
          issue = True
          wrong_rows += 1
          break
    if(issue):
      print(f"Processing row {r+1}... ", end = " ")
      remake_LLM(r + 1)
      for c in range(1, len(rows[0])):
        if(rows[r][c] != "" and rows[r][c] != dummy_rows[r][c]):
          if(c != 14 and c != 15 and c != 19):
            unfixed_rows += 1
            break

  print(f"{wrong_rows} detected incorrect, {unfixed_rows} remain unfixed.")

def fix_LLM_blanks():
  spreadsheet = gc.open('WineList')
  worksheet = spreadsheet.get_worksheet(0) #0 for real data, 1 for dummy
  dummy_worksheet = spreadsheet.get_worksheet(1)
  rows = worksheet.get_all_values()
  dummy_rows = dummy_worksheet.get_all_values()

  wrong_rows = 0
  unfixed_rows = 0

  for r in range (1, len(rows)):
    issue = False;
    for c in range(1, len(rows[0])):
      if(dummy_rows[r][c] == "" and c != 6):
          issue = True
          wrong_rows += 1
          break
    if(issue):
      print(f"Processing row {r+1}... ", end = " ")
      remake_LLM(r + 1)
      for c in range(1, len(rows[0])):
        if(dummy_rows[r][c] == "" and c != 6):
            unfixed_rows += 1
            break

  print(f"{wrong_rows} detected incorrect, {unfixed_rows} remain unfixed.")

update_profiles()
read_profiles()

print(profile_options)
#print(pairing_options)
#print(category_list)
print(specific_tastes)
#fix_LLM_blanks()
#fix_LLM_mistakes()

['https://www.vivino.com/US/en/black-stallion-chardonnay/w/1247400?srsltid=AfmBOorgvYVfNzZcCS8deQHvlDjp5OO0ooAYNPVSVA4lYeL9aQHQqkhD', 'Black Stallion Chardonnay', 'Napa Flats', '8', '21.2', 'Black Stallion', '', 'Napa Valley Chardonnay', 'Napa Valley', 'Chardonnay', '14.5', '2.32', '0.00', '2.74', "['oak', 'microbio', 'non_oak']", "['Pork', 'Vegetarian', 'Poultry', 'Rich fish (salmon, tuna etc)']", '', 'White wine', '4.72', "['oak', 'butter', 'vanilla', 'citrus', 'cream']", '0', 'United States', '1247400']
{'black_fruit', 'dried_fruit', 'non_oak', 'earth', 'oak', 'spices', 'vegetal', 'citrus_fruit', 'red_fruit', 'microbio', 'tree_fruit', 'floral', 'tropical_fruit'}
{'allspice', 'dark chocolate', 'cola', 'yellow apple', 'vanilla', 'grass', 'lime', 'honeysuckle', 'chamomile', 'wild strawberry', 'stone fruit', 'apple', 'red cherry', 'guava', 'smoke', 'melon', 'red currant', 'game', 'lilac', 'butter', 'citrus', 'cassis', 'green mango', 'orange', 'honey', 'balsamic', 'pomegranate', 'tangeri

In [ ]:
import google.generativeai as genai
import ast
from google.colab import userdata

data_structure = ["Bottle Price",	"Winery",	"Year", "Wine Style",	"Region",	"Grape Varieties",	"ABV",	"Dry/Sweet",	"Tannin",	"Soft/Acidic",	"Flavor Profile",	"Pairings", "Rating", "Category", "Light/Bold", "Profile Specifics", "Fizziness", "Country"]


GOOGLE_KEY= userdata.get('GOOGLE_KEY')
genai.configure(api_key=GOOGLE_KEY)

model = genai.GenerativeModel('models/gemini-1.5-flash')
example_vect = get_data("https://www.vivino.com/US/en/messina-hof-private-reserve-double-barrel-shiraz/w/4703?srsltid=AfmBOopG3wZNDhXQSndQJF59kZVJPx21RD6GevxYjuvs-eDfJkhk1cwI")

def fill_gaps(wine_name, wine_vect): #add list of profiles and flavors to draw from
  response = model.generate_content("I am representing wines as a list with the following features in the following order: " + str(data_structure) + ". "
                                    "Bottle price is in USD. Dry/Sweet, Tannic, Soft/Acidic, Rating, Light/Bold, and Fizziness are all rated on a scale from 1-5. "
                                    "The flavor profile is a list of 3 high-level taste categories from the following list: " + str(profile_options) + ". "
                                    "The pairings section is a list of 3-5 food categories that pair well with this wine from the following list: " + str(pairing_options) + ". "
                                    "The category is one of the following: " + str(category_list) + ". "
                                    "Profile specifics is a list of 5 more specific flavors found in the wine from the following list: " + str(specific_tastes) + ". Do not include any flavors not found in that list. If the flavor is not in that list exactly, do not include it."
                                    "This vector is an example of the formatting of a fully filled out wine vector: " + str(example_vect) + ". The vector you return should have the same format."
                                    "Given the following vector that represents a the wine " + wine_name + ", fill in all of the missing values except for year. Leave none blank except year."
                                    "Make sure you put in a value for wine bottle price and rating based on all information you have. Do not leave these blank."
                                    "If there is already a value in the vector, DO NOT CHANGE IT. Do not round values or change anything if they are already filled out. "
                                    "If there is already a number in the vector I give you, the exact same number should be in the vector you return."
                                    "If there is already any information in the vector I give you, the exact same information should be in the vector you return."
                                    "Return only the vector with the filled in values and nothing else. " + str(wine_vect)
  )

  #todo format checking
  try:
    output_list = ast.literal_eval(response.text)
    if(len(output_list) != len(data_structure)):
      print("ERROR PARSING LLM RESPONSE: WRONG LENGTH")
      return([])
    if((output_list[7] != "" and (float(output_list[7]) > 5 or float(output_list[7]) < 0)) or (output_list[8] != "" and (float(output_list[8]) > 5 or float(output_list[8]) < 0)) or (output_list[9] != "" and (float(output_list[9]) > 5 or float(output_list[9]) < 0)) or (output_list[12] != "" and (float(output_list[12]) > 5 or float(output_list[12]) < 0)) or (output_list[14] != "" and (float(output_list[14]) > 5 or float(output_list[14]) < 0)) or (output_list[16] != "" and (float(output_list[16]) > 5 or float(output_list[16]) < 0))):
      print("ERROR PARSING LLM RESPONSE: ONE OR MORE VALUES OUT OF RANGE")
      return([])
    if(output_list[13] not in category_list):
      print("ERROR PARSING LLM RESPONSE: CATEGORY NOT FOUND " + output_list[13])
      return([])
    for f in ast.literal_eval(output_list[10]):
      if(f not in profile_options):
        print("ERROR PARSING LLM RESPONSE: PROFILE NOT FOUND " + output_list[10])
        return([])
    for p in ast.literal_eval(output_list[11]):
      if(p not in pairing_options):
        print("ERROR PARSING LLM RESPONSE: PAIRING NOT FOUND " + p)
        return([])
    for f in ast.literal_eval(output_list[15]):
      if(f not in specific_tastes):
        print("ERROR PARSING LLM RESPONSE: SPECIFIC PROFILE NOT FOUND " + f)
        return([])
  except Exception as e:
    print("ERROR PARSING LLM RESPONSE: " + str(e))
    return([])
  output_list = ast.literal_eval(response.text)
  return(output_list)

fill_gaps("Old Texas Roundup", get_data("https://www.vivino.com/US/en/messina-hof-nimitz-old-texas-roundup-dry-red-blend/w/13116715?srsltid=AfmBOop6FZjTWtE2vp4vYh6z50l_rrE_gQ11hdGfrkAb1inzgvlz2fYt"))

3 item(s) not found; ['Bottle Price', 'Year', 'Rating']5 item(s) not found; ['Bottle Price', 'Year', 'Flavor Profile', 'Rating', 'Profile Specifics']ERROR PARSING LLM RESPONSE: invalid syntax (<unknown>, line 1)


[]

In [ ]:
remake_LLM(268)

Processing row 268...  9 item(s) not found; ['Year', 'Wine Style', 'Grape Varieties', 'Dry/Sweet', 'Tannin', 'Soft/Acidic', 'Pairings', 'Light/Bold', 'Fizziness']
['17.34', 'Messina Hof', '', '', 'Texas', '', '18', '', '', '', "['oak', 'dried_fruit', 'non_oak']", '', '4', 'Fortified Wine', '', "['chocolate', 'oak', 'vanilla', 'raisin', 'fig']", '', 'United States']
['17.34', 'Messina Hof', '', 'Port', 'Texas', '', '18', 3, 4, 2, "['oak', 'dried_fruit', 'non_oak']", "['Sweet desserts', 'Mature and hard cheese', 'Cured Meat']", '4', 'Fortified Wine', 4, "['chocolate', 'oak', 'vanilla', 'raisin', 'fig']", 0, 'United States']



In [ ]:
update_profiles_LLM()
#fix_LLM_blanks()
#fix_LLM_mistakes()

In [ ]:
#download winefolly images
#upload code from https://stackoverflow.com/questions/50647677/upload-file-from-colab-to-google-drive-folder
from google.colab import drive
import shutil
import requests
import xml.etree.ElementTree as ET

drive.mount('/content/drive/', force_remount=True)

def get_taste_image(taste, filename):
  url = 'https://media.winefolly.com/wines/icons/flavors/' + taste + '.svg'
  svg = requests.get(url).text
  path = "/content/drive/Shareddrives/AggieWineGlass/Photos/TastingPhotos/" + filename + ".svg"
  if("Error" in svg):
    print("Error getting svg for " + taste)
    return False, taste
  if("g id=" in svg):
    soup = BeautifulSoup(svg, features="xml")
    if(soup.find('g', attrs={'id':taste}) == None):
      print("Error getting svg for " + taste)
      return False, taste
    for group in soup.find_all('g', attrs={'id':taste}):
      with open(path, 'w') as file:
        file.write('<svg viewBox="0 0 26 26" xmlns="http://www.w3.org/2000/svg">' + str(group) + '</svg>')
        print("Saved " + filename)
        return True, filename
  else:
    with open(path, 'w') as file:
      file.write(svg)
      print("Saved " + filename)
      return True, filename

#print(get_taste_image("black-cherry", "black-cherry")[0])
#drive.mount('/content/drive/')

all_tastes = profile_options.union(specific_tastes)
found_tastes = set()
unfound_tastes = set()

for taste in all_tastes:
  taste_edit = taste.replace(" ", "-")
  found, taste = get_taste_image(taste_edit, taste)
  if(found):
    found_tastes.add(taste)
  else:
    unfound_tastes.add(taste)
#main svg thing, subgroup, filename
#found, taste = get_taste_image_subpage('tropical', 'tropical_fruit', 'tropical')
#get_taste_image('cherry', 'cherry')

#print(unfound_tastes)

#taste, filename
#get_taste_image('lime-peel', 'lemon zest')

#tropical: ['ruler', 'lilac_1_', 'violet', 'violet_copy', 'anise_1_', '_x35_-spice_1_', 'rose', 'hibiscus_1_', 'acacia_1_', 'sage', 'pepper_black', 'pepper_white', 'bell_pepper', 'cinnamon', 'dill', 'perfume', 'pepper_red', 'dried_herbs', 'tomato_leaf', 'honeysuckle_1_', 'jasmine_1_', 'geranium_1_', 'chamomile', 'gooseberry', 'peony', 'lemongrass', 'cut_grass', 'rhubarb_1_', 'chervil-cilantro', 'dafodil_1_', 'citris-blossom', 'green_bean', 'eucalyptus', 'mint', 'celery', 'tarragon', 'fennel', 'oregano', 'lemon', 'lime', 'lemon_peel', 'orange', 'tangerine', 'grapefruit', 'quince', 'asian_pear', 'green_pear_copy', 'yellow_pear', 'red_apple', 'Layer_50', 'peach', 'white_peach', 'nectartine', 'apricot_1_', 'jackfruit', 'tropical_fruit']
get_taste_image('yellow-apple', 'test')


Mounted at /content/drive/
Saved pineapple
Error getting svg for allspice
Error getting svg for dark-chocolate
Error getting svg for cola
Error getting svg for yellow-apple
Error getting svg for lychee
Saved vanilla
Error getting svg for lime-zest
Saved almond
Error getting svg for nutty
Error getting svg for grass
Error getting svg for lime
Saved cedar
Error getting svg for red-plum
Saved cocoa
Error getting svg for butterscotch
Error getting svg for brioche
Error getting svg for floral
Error getting svg for honeysuckle
Error getting svg for tropical_fruit
Error getting svg for nutmeg
Saved chamomile
Error getting svg for spices
Error getting svg for nectarine
Error getting svg for mango
Error getting svg for wild-strawberry
Error getting svg for stone-fruit
Error getting svg for honeydew-melon
Error getting svg for red_fruit
Error getting svg for elderflower
Error getting svg for coffee
Saved apricot
Saved apple
Error getting svg for red-cherry
Error getting svg for tropical
Error ge

(False, 'yellow-apple')

In [ ]:
#gpt code
#taste, filename
#get_taste_image('grass', 'grass')
import os

num_images = 0
image_set = set()

drive.mount('/content/drive/', force_remount=True)
for filename in os.listdir('/content/drive/Shareddrives/AggieWineGlass/Photos/TastingPhotos/'):
  num_images += 1
  image_set.add(filename.replace(".svg", ""))

all_tastes = profile_options.union(specific_tastes)

print(all_tastes - image_set)
print(image_set - all_tastes)

image_set = set()
for filename in os.listdir('/content/drive/Shareddrives/AggieWineGlass/Photos/PairingPhotos/'):
  image_set.add(filename.replace(".png", ""))

print(pairing_options - image_set)
print(image_set - pairing_options)

Mounted at /content/drive/
set()
set()
set()
set()
